In [ ]:
import numpy as np
import os
import torch
#from neuromaps.datasets import fetch_fsaverage
#from surfplot import Plot
import scipy.io
import warnings
from nilearn import datasets
from nilearn import plotting
from PIL import Image

import matplotlib.pyplot as plt

from spacetorch.feature_extractor import FeatureExtractor
from torch.utils.data import Dataset, DataLoader
from spacetorch.datasets import DatasetRegistry
from einops import reduce, rearrange
from load_model import get_closest_factors

device = ('cuda' if torch.cuda.is_available() else 'cpu')
from torch.utils.data import DataLoader, Dataset

from pathlib import Path
from torchvision.transforms import transforms
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

from load_model import *
from spacetorch.feature_extractor import FeatureExtractor
import torchvision


# Correlation

In [ ]:

DUMMY_LABEL = 0

class ImageDataset(Dataset):
    def __init__(self, imgs_paths, idxs, transform):
        self.imgs_paths = np.array(imgs_paths)[idxs]
        self.transform = transform

    def __len__(self):
        return len(self.imgs_paths)

    def __getitem__(self, idx):
        # Load the image
        img_path = self.imgs_paths[idx]
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img).to(device)
        return img,DUMMY_LABEL






def ROI_selection(hemisphere, roi):
    subject_dir = '/home/xinyuq/projects/def-bashivan/xinyuq/kernel_avpool/data/datasets/algonauts_2023_tutorial_data/subj04'
    # Define the ROI class based on the selected ROI
    if roi in ["V1v", "V1d", "V2v", "V2d", "V3v", "V3d", "hV4"]:
        roi_class = 'prf-visualrois'
    elif roi in ["EBA", "FBA-1", "FBA-2", "mTL-bodies"]:
        roi_class = 'floc-bodies'
    elif roi in ["OFA", "FFA-1", "FFA-2", "mTL-faces", "aTL-faces"]:
        roi_class = 'floc-faces'
    elif roi in ["OPA", "PPA", "RSC"]:
        roi_class = 'floc-places'
    elif roi in ["OWFA", "VWFA-1", "VWFA-2", "mfs-words", "mTL-words"]:
        roi_class = 'floc-words'
    elif roi in ["early", "midventral", "midlateral", "midparietal", "ventral", "lateral", "parietal"]:
        roi_class = 'streams'

    # Load the ROI brain surface maps
    roi_class_dir = os.path.join(subject_dir, 'roi_masks', hemisphere[0]+'h.'+roi_class+'_fsaverage_space.npy')
    roi_map_dir = os.path.join(subject_dir, 'roi_masks','mapping_'+roi_class+'.npy')
    fsaverage_roi_class = np.load(roi_class_dir)
    roi_map = np.load(roi_map_dir, allow_pickle=True).item()

    roi_mapping = list(roi_map.keys())[list(roi_map.values()).index(roi)]
    fsaverage_roi = np.asarray(fsaverage_roi_class == roi_mapping, dtype=int)
    return fsaverage_roi


def point_inside_polygon(x, y, poly):
    n = len(poly)
    inside = False
    p1x, p1y = poly[0]
    for i in range(n + 1):
        p2x, p2y = poly[i % n]
        if y > min(p1y, p2y):
            if y <= max(p1y, p2y):
                if x <= max(p1x, p2x):
                    if p1y != p2y:
                        xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                    if p1x == p2x or x <= xinters:
                        inside = not inside
        p1x, p1y = p2x, p2y
    return inside


def select_intermediate_patch(hemisphere,roi_1,roi_2,rectangle_size_x=30,rectangle_size_y=5,adjust_x_1=0,adjust_y_1=0):

    path_info = "/home/xinyuq/projects/def-bashivan/xinyuq/kernel_avpool/data/datasets/algonauts_2023_tutorial_data/"
    subject_dir =  os.path.join(path_info, 'subj04/')
    subject_info = os.path.join(subject_dir, 'roi_masks/')

    fsaverage_roi_patch1 = ROI_selection(hemisphere, roi_1)
    fsaverage_roi_patch2 = ROI_selection(hemisphere, roi_2)


    rh_fsaverage_x = scipy.io.loadmat(path_info+f'{hemisphere[0]}h_x.mat')[f'x_{hemisphere[0]}h']
    rh_fsaverage_y = scipy.io.loadmat(path_info+f'{hemisphere[0]}h_y.mat')[f'y_{hemisphere[0]}h']
    rh_fsaverage_ind = scipy.io.loadmat(path_info+f'{hemisphere[0]}h_ind.mat')[f'ind_{hemisphere[0]}h']

    temp_rh = np.zeros((len(fsaverage_roi_patch1),1))
    temp_rh[rh_fsaverage_ind] = 1
    temp_rh = temp_rh.reshape(len(temp_rh),)

    X_val = np.full((len(fsaverage_roi_patch1),), np.nan)
    Y_val = np.full((len(fsaverage_roi_patch1),), np.nan)

    X_val[temp_rh == 1,] = rh_fsaverage_x.reshape(np.size(rh_fsaverage_x,axis=1),)
    Y_val[temp_rh == 1,] = rh_fsaverage_y.reshape(np.size(rh_fsaverage_y,axis=1),)

    rh_patch1_data_with_xy = np.full((len(fsaverage_roi_patch1),3), np.nan)# 0: data; 1: x value; 2: y_value
    rh_patch1_data_with_xy[:,0] = fsaverage_roi_patch1
    rh_patch1_data_with_xy[:,1] = X_val
    rh_patch1_data_with_xy[:,2] = Y_val

    rh_patch2_data_with_xy = np.full((len(fsaverage_roi_patch2),3), np.nan)# 0: data; 1: x value; 2: y_value
    rh_patch2_data_with_xy[:,0] = fsaverage_roi_patch2
    rh_patch2_data_with_xy[:,1] = X_val
    rh_patch2_data_with_xy[:,2] = Y_val

    # patch 1
    # Masking the values based on the first dimension
    mask = rh_patch1_data_with_xy[:, 0] == 1
    # Apply the mask to your data
    filtered_data = rh_patch1_data_with_xy[mask]
    # Reshape the filtered data to match the new shape
    new_shape = (filtered_data.shape[0],) + rh_patch1_data_with_xy.shape[1:]
    data_patch1 = filtered_data.reshape(new_shape)
    print('data1', data_patch1.shape)

    # patch2
    # Masking the values based on the first dimension
    mask = rh_patch2_data_with_xy[:, 0] == 1
    # Apply the mask to your data
    filtered_data = rh_patch2_data_with_xy[mask]
    # Reshape the filtered data to match the new shape
    new_shape = (filtered_data.shape[0],) + rh_patch2_data_with_xy.shape[1:]
    data_patch2 = filtered_data.reshape(new_shape)

    # Find the mean x and y of the two patches
    X_patch1_mean = np.nanmean(data_patch1[:,1])
    Y_patch1_mean = np.nanmean(data_patch1[:,2])
    X_patch2_mean = np.nanmean(data_patch2[:,1])
    Y_patch2_mean = np.nanmean(data_patch2[:,2])
    ###############################################################################
    # Define the coordinates of the two patches
    patch1_x, patch1_y = X_patch1_mean, Y_patch1_mean
    patch2_x, patch2_y = X_patch2_mean, Y_patch2_mean

    # Calculate the direction vector of the line connecting the two points
    direction_vector = np.array([patch2_x - patch1_x, patch2_y - patch1_y])
    direction_vector /= np.linalg.norm(direction_vector)  # Normalize the direction vector

    # Calculate the coordinates of the square's vertices based on the direction vector
    center_x = (patch1_x + patch2_x) / 2
    center_y = (patch1_y + patch2_y) / 2


    offset_vector_x = rectangle_size_x / 2 * np.array([direction_vector[1], direction_vector[0]])
    offset_vector_y = rectangle_size_y / 2 * np.array([-direction_vector[0], direction_vector[1]])

    # Calculate the coordinates of the rectangle's vertices
    rectangle_vertices = [
        [center_x + offset_vector_x[0] + offset_vector_y[0] - adjust_x_1, center_y + offset_vector_x[1] + offset_vector_y[1] - adjust_y_1],
        [center_x - offset_vector_x[0] + offset_vector_y[0] - adjust_x_1, center_y - offset_vector_x[1] + offset_vector_y[1]- adjust_y_1],
        [center_x - offset_vector_x[0] - offset_vector_y[0] - adjust_x_1, center_y - offset_vector_x[1] - offset_vector_y[1]- adjust_y_1],
        [center_x + offset_vector_x[0] - offset_vector_y[0] - adjust_x_1, center_y + offset_vector_x[1] - offset_vector_y[1]- adjust_y_1]
    ]
    rectangle_vertices = np.array(rectangle_vertices)
    ###############################################################################
    # Generate a grid of points within the rectangle's bounding box
    x_grid = np.linspace(min(rectangle_vertices[:, 0]), max(rectangle_vertices[:, 0]), num=100)
    y_grid = np.linspace(min(rectangle_vertices[:, 1]), max(rectangle_vertices[:, 1]), num=100)
    X, Y = np.meshgrid(x_grid, y_grid)

    points_within_rectangle = np.column_stack((X_val.flatten(),Y_val.flatten()))

    filtered_points = [point for point in points_within_rectangle if point_inside_polygon(point[0], point[1], rectangle_vertices)]
    filtered_points = np.array(filtered_points)

    ROIs = rh_patch1_data_with_xy + rh_patch2_data_with_xy
    other_patches = rh_patch1_data_with_xy + rh_patch2_data_with_xy

    intermediate_patch = np.zeros(ROIs.shape)
    for i in range(len(filtered_points)):
        # Find indices where filtered point matches X_val and Y_val
        matching_indices = np.where((filtered_points[i, 0] == X_val) & (filtered_points[i, 1] == Y_val))[0]
        # Set the corresponding rows in mask_rect to 2
        ROIs[matching_indices, :] = 2
        intermediate_patch[matching_indices, :] = 2

    return ROIs,intermediate_patch,other_patches,fsaverage_roi_patch1,fsaverage_roi_patch2



def get_intermediate_fMRI(hemisphere,ROI_fsaverage, subject_dir, length, fmri_dir):

    roi_dir = os.path.join(subject_dir, 'roi_masks', hemisphere[0] + 'h.all-vertices_fsaverage_space.npy')

    fsaverage_all_vertices = np.load(roi_dir)

    temp =  np.zeros([length, fsaverage_all_vertices.shape[0]])
    #print(len(train_img_list))
    fmri_value = np.load(os.path.join(fmri_dir, f'{hemisphere[0]}h_training_fmri.npy'))

    valid_indices = np.where(fsaverage_all_vertices == 1)[0]
    #print(temp.shape)
    #print(valid_indices.shape)
    #print(fmri_value.shape)
    temp[:, valid_indices] = fmri_value[:, :len(valid_indices)]

    roi_idx = np.where(ROI_fsaverage)[0]
    fmri_value = temp[:,roi_idx]
    print(fmri_value.shape)
    return fmri_value



def plot_correlations(features,fMRI_value, patch):

    num_channles = features.shape[1]
    avg_feature_activations = features #np.mean(features, axis=(2,3)) # Shape: (529, 7, 7)

    # Calculate the average fMRI data across images
    avg_fMRI_data = np.mean(fMRI_value,axis=(1))  # Shape: (266,)

    correlations = np.zeros(num_channles)

    for channel in range(num_channles):
        correlations[channel] = np.corrcoef(
            avg_feature_activations[:,channel],
            avg_fMRI_data
        )[0, 1]
    x_dim,y_dim = get_closest_factors(num_channles)
    x_dim = int(x_dim/2)
    y_dim = int(y_dim*2)

    correlation_grid = correlations.reshape(y_dim, x_dim)

    plt.figure(figsize=(10, 40))
    #plt, ax = plt.subplots(figsize=(10, 40))
    heatmap = plt.imshow(correlation_grid.reshape(y_dim,x_dim), cmap='coolwarm', interpolation='nearest',vmin=-0.5, vmax=0.5, aspect="auto")
    plt.colorbar(heatmap, aspect=40)
    plt.title(f'unit correlation for {patch} patch')

    plt.xticks(range(x_dim))
    plt.yticks(range(y_dim))
    plt.show()
    #plt.gca().spines[['left','bottom','right', 'top']].set_visible(False)
    plt.gca().axes.get_xaxis().set_visible(False)
    plt.gca().axes.get_yaxis().set_visible(False)

    plt.savefig(f"inter/{patch}.png")#, format='pdf')
    plt.savefig(f"inter/{patch}.pdf", format='pdf')
    plt.clf()

    top_52_indices = np.unravel_index(np.argsort(correlation_grid, axis=None)[-10:], correlation_grid.shape)

    top_52_matrix = np.zeros_like(correlation_grid)

    for index in zip(*top_52_indices):
        top_52_matrix[index] = 1
    return top_52_matrix.reshape(y_dim, x_dim)



In [ ]:

def main():

    hemisphere = 'right'
    #ROIs_2,intermediate_2,other_patches, f1, f2 = select_intermediate_patch(hemisphere=hemisphere,roi_1='FFA-2',roi_2='PPA',rectangle_size_x=5,rectangle_size_y=5,adjust_x_1=5,adjust_y_1=0)
    ROIs_2,intermediate_2,other_patches, f1, f2 = select_intermediate_patch(hemisphere=hemisphere,roi_1='FFA-2',roi_2='PPA',rectangle_size_x=5,rectangle_size_y=5,adjust_x_1=0,adjust_y_1=0)


    fmri_dir = os.path.join('/home/xinyuq/projects/def-bashivan/xinyuq/kernel_avpool/data/datasets/algonauts_2023_tutorial_data/subj04', 'training_split', 'training_fmri')
    lh_fmri = np.load(os.path.join(fmri_dir, 'lh_training_fmri.npy'))
    rh_fmri = np.load(os.path.join(fmri_dir, 'rh_training_fmri.npy'))
    path_info = "/home/xinyuq/projects/def-bashivan/xinyuq/kernel_avpool/data/datasets/algonauts_2023_tutorial_data/"
    subject_dir =  os.path.join(path_info, 'subj04/')
    subject_info = os.path.join(subject_dir, 'roi_masks/')

    train_img_dir  = os.path.join(subject_dir, 'training_split', 'training_images')

    # Create lists will all training and test image file names, sorted
    train_img_list = os.listdir(train_img_dir)
    train_img_list.sort()


    batch_size = 300
    train_imgs_paths = sorted(list(Path(train_img_dir).iterdir()))

     # Calculate how many stimulus images correspond to 90% of the training data
    num_train = int(np.round(len(train_img_list) / 100 * 90))
    # Shuffle all training stimulus images
    idxs = np.arange(len(train_img_list))
    np.random.shuffle(idxs)
    # Assign 90% of the shuffled stimulus images to the training partition,
    # and 10% to the test partition
    idxs_train, idxs_val = idxs[:num_train], idxs[num_train:]


    intermediate_fMRI= get_intermediate_fMRI( hemisphere=hemisphere,ROI_fsaverage=intermediate_2[:,0], subject_dir=subject_dir, length=len(train_img_list), fmri_dir=fmri_dir)[idxs_train]

    print('fmri', intermediate_fMRI)
    face_fMRI= get_intermediate_fMRI( hemisphere=hemisphere,ROI_fsaverage=f1, subject_dir=subject_dir, length=len(train_img_list), fmri_dir=fmri_dir)[idxs_train]
    place_fMRI= get_intermediate_fMRI( hemisphere=hemisphere,ROI_fsaverage=f2, subject_dir=subject_dir, length=len(train_img_list), fmri_dir=fmri_dir)[idxs_train]





    train_imgs_dataloader = DataLoader(
        ImageDataset(train_imgs_paths, idxs_train, transform),
        batch_size=batch_size
    )

    model = load_model(pool_type='gaussian', kap_kernelsize=0.23, continuous=True, local_conv=False, expname='gaussian_0.23_continuous_prog_t', epoch=100, sel_range=10)
    #features= FeatureExtractor(train_imgs_dataloader, 32).extract_features(model, [model.layer4[0].bn1, model.layer4[0].bn2, model.layer4[1].bn1, model.layer4[1].bn2], return_inputs_and_labels=False)
    features= FeatureExtractor(train_imgs_dataloader, 32).extract_features(model, [model.layer4[0].conv1, model.layer4[0].conv2, model.layer4[1].conv1, model.layer4[1].conv2], return_inputs_and_labels=False)
    #features= FeatureExtractor(train_imgs_dataloader, 32).extract_features(model, [model.layer3[0].conv1, model.layer3[0].conv2, model.layer3[1].conv1, model.layer3[1].conv2], return_inputs_and_labels=False)
    features_1, features_2, features_3, features_4 = features
    avg_features_1 = reduce(features_1, 'b c h w -> b c', 'mean')
    avg_features_2 = reduce(features_2, 'b c h w -> b c', 'mean')
    avg_features_3 = reduce(features_3, 'b c h w -> b c', 'mean')
    avg_features_4 = reduce(features_4, 'b c h w -> b c', 'mean')

    avg_features = np.concatenate((avg_features_1, avg_features_2, avg_features_3, avg_features_4), axis=1)

    top_52_matrix_3_f = plot_correlations(features=avg_features,fMRI_value=face_fMRI, patch='face')
    top_52_matrix_3_inter = plot_correlations(features=avg_features,fMRI_value=intermediate_fMRI, patch='intermediate')
    top_52_matrix_3_p = plot_correlations(features=avg_features,fMRI_value=place_fMRI, patch='place')

    three_in_one(top_52_matrix_3_inter,top_52_matrix_3_f, top_52_matrix_3_p)
